In [28]:
import polars as pl

kp = pl.read_parquet('kjoretoy_preppet')

df_ant_el_2022 = kp.filter(
    (pl.col('tekn_reg_f_g_n').cast(str).str.contains('2022')) 
    & (pl.col('tekn_drivstoff') == '5')
)

print(f"Antall Elbiler registrert i 2022 er: {df_ant_el_2022.height}")

Antall Elbiler registrert i 2022 er: 37249
